In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, InputLayer, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
# Image specifications
img_height = 128
img_width = 128
batch_size = 64

In [4]:
# Prepare ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [6]:
# Define the model architecture
model = Sequential([
    InputLayer(input_shape=(img_height, img_width, 3)),  # 3 channels for RGB images
    Conv2D(32, (3, 3), activation='relu'),  # First convolution layer
    MaxPooling2D((2, 2)),  # First max pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Second convolution layer
    MaxPooling2D((2, 2)),  # Second max pooling layer
    Conv2D(128, (3, 3), activation='relu'),  # Third convolution layer
    MaxPooling2D((2, 2)),  # Third max pooling layer
    Flatten(),  # Flatten the output to feed into a DNN
    Dense(128, activation='relu'),  # Dense layer after convolution layers
    Dropout(0.5),  # Dropout layer to reduce overfitting
    Dense(1, activation='sigmoid')  # Output layer with one neuron using sigmoid activation
])

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [7]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [8]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
number_of_epochs = 10

In [10]:
# Train the model
history = model.fit(
    train_generator,
    epochs = number_of_epochs,
    validation_data=validation_generator
)

Epoch 1/10


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 878ms/step - accuracy: 0.4535 - loss: 1.0332 - val_accuracy: 0.4286 - val_loss: 0.8285
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 464ms/step - accuracy: 0.5453 - loss: 0.7320 - val_accuracy: 0.4286 - val_loss: 0.6977
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 487ms/step - accuracy: 0.5861 - loss: 0.6899 - val_accuracy: 0.4286 - val_loss: 0.6948
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 458ms/step - accuracy: 0.5809 - loss: 0.6891 - val_accuracy: 0.4286 - val_loss: 0.7064
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 499ms/step - accuracy: 0.5762 - loss: 0.6846 - val_accuracy: 0.4286 - val_loss: 0.7177
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 464ms/step - accuracy: 0.5889 - loss: 0.6800 - val_accuracy: 0.4286 - val_loss: 0.7008
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 525ms/step - accuracy: 0.5812 - loss: 0.6840 - val_accuracy: 0.4286 - val_loss: 0.7026
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 473ms/step - accuracy: 0.6110 - loss: 0.6742 - val_accuracy: 0.4286 - val_loss: 0.7137
Epo

In [11]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 983ms/step - accuracy: 0.4286 - loss: 0.7202
Validation accuracy: 0.4285714328289032


In [12]:
# Save the model, optional
model.save(f'Pth_Files/CNN_model_{number_of_epochs}.h5')